In [1]:
# Pick response for supervised prob
response = "quality"

# df[response] = df[response].asfactor()

In [2]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/rocket/anaconda2/h2o_jar/h2o.jar
  Ice root: /tmp/tmp48PJw1
  JVM stdout: /tmp/tmp48PJw1/h2o_rocket_started_from_python.out
  JVM stderr: /tmp/tmp48PJw1/h2o_rocket_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,"1 year, 2 months and 23 days !!!"
H2O cluster name:,H2O_from_python_rocket_483mll
H2O cluster total nodes:,1
H2O cluster free memory:,3.464 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
data = h2o.import_file(path="../datasets/wine.csv")
data.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:1599
Cols:12




,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,real,real,real,real,real,real,real,real,real,real,real,int
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0
mean,8.3196372733,0.527820512821,0.270975609756,2.53880550344,0.0874665415885,15.8749218261,46.4677923702,0.996746679174,3.31111319575,0.658148843027,10.4229831144,5.63602251407
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9,8.0
sigma,1.74109631813,0.179059704154,0.194801137405,1.40992805951,0.0470653020101,10.4601569698,32.8953244783,0.00188733395384,0.154386464904,0.16950697959,1.06566758185,0.807569439735
zeros,0,0,132,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0


In [4]:
# Define predictors
predictors = data.columns
del predictors[-1]
print predictors

[u'fixed acidity', u'volatile acidity', u'citric acid', u'residual sugar', u'chlorides', u'free sulfur dioxide', u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol']


In [5]:
# Split for training
train, valid, test = data.split_frame(
    ratios = [0.6, 0.2],
    seed=4567,
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [6]:
# Defualt gbm tree
gbm = H2OGradientBoostingEstimator()
gbm.train(
    x                 = predictors,
    y                 = response,
    training_frame    = train,
    ntrees            = 100,
    max_depth         = 3,
    learn_rate        = 0.2)

# Show model summary
print gbm

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1517075776023_1
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,14899.0,5.0,5.0,5.0,8.0,29.0,18.7




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.166621537974
RMSE: 0.408193015586
MAE: 0.314239536759
RMSLE: 0.0640642920971
Mean Residual Deviance: 0.166621537974
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-01-27 12:56:18,0.016 sec,0.0,0.8068536,0.6880342,0.6510127
,2018-01-27 12:56:18,0.140 sec,1.0,0.7692143,0.6554699,0.5916906
,2018-01-27 12:56:18,0.191 sec,2.0,0.7350752,0.6247230,0.5403355
,2018-01-27 12:56:18,0.234 sec,3.0,0.7066001,0.5978307,0.4992836
,2018-01-27 12:56:18,0.293 sec,4.0,0.6811829,0.5738549,0.4640101
---,---,---,---,---,---,---
,2018-01-27 12:56:19,0.882 sec,46.0,0.4150770,0.3211477,0.1722889
,2018-01-27 12:56:19,0.892 sec,47.0,0.4139543,0.3200619,0.1713581
,2018-01-27 12:56:19,0.903 sec,48.0,0.4115153,0.3177601,0.1693449
,2018-01-27 12:56:19,0.911 sec,49.0,0.4105452,0.3166419,0.1685474



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
alcohol,818.1927490,1.0,0.3344971
sulphates,379.1015320,0.4633401,0.1549859
volatile acidity,220.8098755,0.2698751,0.0902724
citric acid,220.6299438,0.2696552,0.0901989
total sulfur dioxide,178.9376678,0.2186987,0.0731541
pH,128.1551209,0.1566319,0.0523929
chlorides,125.9714661,0.1539631,0.0515002
free sulfur dioxide,104.1572418,0.1273016,0.0425820
fixed acidity,95.9387970,0.1172570,0.0392221
residual sugar,89.0492249,0.1088365,0.0364055


In [7]:
perf = gbm.model_performance(valid)
perf


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.379919781468
RMSE: 0.616376331042
MAE: 0.475474731432
RMSLE: 0.0949689605206
Mean Residual Deviance: 0.379919781468


In [8]:
cv_gbm = H2OGradientBoostingEstimator(nfolds = 4, seed = 0xDEFCA)
cv_gbm.train(x=predictors, y=response, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:
# Show detailed summary of cross validation metrics
# Gives better idea of variance between the folds
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
print(cv_summary) # Full Summary
print(cv_summary.iloc[4])

                            mean           sd  cv_1_valid  cv_2_valid  \
0                mae  0.47932813  0.008474927   0.4926777  0.48939338   
1                mse  0.40390328  0.014636368  0.41108084  0.43260002   
2                 r2  0.37745756  0.031093763  0.39250833  0.30729967   
3  residual_deviance  0.40390328  0.014636368  0.41108084  0.43260002   
4               rmse   0.6353252  0.011514212  0.64115584  0.65772337   
5              rmsle   0.0985848  0.002430354  0.09727838  0.10351618   

    cv_3_valid  cv_4_valid  
0     0.470857   0.4643844  
1   0.39584747  0.37608477  
2    0.4281144  0.38190776  
3   0.39584747  0.37608477  
4    0.6291641   0.6132575  
5  0.099479504  0.09406515  
                     rmse
mean            0.6353252
sd            0.011514212
cv_1_valid     0.64115584
cv_2_valid     0.65772337
cv_3_valid      0.6291641
cv_4_valid      0.6132575
Name: 4, dtype: object


In [10]:
# Get cross-validated AUC through scoring combined holdout predictions.
perf_cv = cv_gbm.model_performance(xval=True)
perf_cv


ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.403819205689
RMSE: 0.635467706252
MAE: 0.479013859317
RMSLE: 0.0987077432725
Mean Residual Deviance: 0.403819205689


In [11]:
gbm_lucky = H2OGradientBoostingEstimator(
    # more trees the better if learning rate small
    # use 'more than enough' since we have early stop
    ntrees = 10000,
    
    # smaller learning rate is better
    learn_rate = 0.01,
    
#     Early stopping once validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "auto",
    
#     Sample 80% of columns per split
    sample_rate = 0.8,
    
#     fix random number generator seed
    seed = 1234,
#     score every 10 trees
    score_tree_interval = 10
)

gbm_lucky.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
perf_lucky = gbm_lucky.model_performance(valid)
perf_lucky


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.365893823401
RMSE: 0.604891579873
MAE: 0.460263083689
RMSLE: 0.0929994150877
Mean Residual Deviance: 0.365893823401


In [13]:
h2o.shutdown()

H2O session _sid_9384 closed.
